In [1]:
# Dependencies
import pandas as pd

# Save path to data set in a variable
file = "Resources/purchase_data.csv"

# Use Pandas to read data
purchase_data = pd.read_csv(file)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [2]:
#Total Number of Players
players = purchase_data["SN"].unique()
players_count = len(players)

pCount_table = pd.DataFrame({"Total Players": [players_count]})
pCount_table

,Total Players
0,576


In [3]:
### Purchasing Analysis (Total)
#Number of Unique Items
items_count = len(purchase_data["Item ID"].unique())

# Average Purchase Price
avg_price = purchase_data["Price"].mean()

# Total Number of Purchases
num_purchases = len(purchase_data["Price"])

# Total Revenue
total_revenue = purchase_data["Price"].sum()

purchasing_analysis = pd.DataFrame({"Number of Unique Items" : [items_count],
                                    "Average Price" : [avg_price],
                                    "Number of Purchases" : [num_purchases],
                                    "Total Revenue" : [total_revenue]})
purchasing_analysis = purchasing_analysis.round(2)

purchasing_analysis["Average Price"] = purchasing_analysis["Average Price"].map("${0:,.2f}".format)
purchasing_analysis["Total Revenue"] = purchasing_analysis["Total Revenue"].map("${0:,.2f}".format)

purchasing_analysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


In [4]:
### Gender Demographics
# Percentage and Count of Male Players
# Percentage and Count of Female Players
# Percentage and Count of Other / Non-Disclosed

gender_info = purchase_data.loc[:, ("SN", "Gender")]

gender_info.drop_duplicates(subset = ["SN"], keep = "last", inplace = True)

gender_counts = gender_info.groupby("Gender").size().reset_index(name='Total Count')
gender_percent = (gender_counts["Total Count"]/players_count)*100
gender_counts["Percentage of Players"] = gender_percent.map("{0:,.2f}%".format)

gender_counts

,Gender,Total Count,Percentage of Players
0,Female,81,14.06%
1,Male,484,84.03%
2,Other / Non-Disclosed,11,1.91%


In [5]:
### Purchasing Analysis (Gender)
#Purchase Count
purchases_by_gender = purchase_data.groupby(["Gender"]).size().reset_index(name="Purchase Count")

In [6]:
gender_purchases_df = pd.DataFrame(purchases_by_gender)

In [7]:
#Average Purchase Price
avg_purchases_by_gender = purchase_data.groupby(["Gender"]).mean()
avg_purchases_by_gender = avg_purchases_by_gender.rename(
    columns={"Price" : "Average Purchase Price"})

In [8]:
#Total Purchase Value
tot_purchases_by_gender = purchase_data.groupby(["Gender"]).sum()
tot_purchases_by_gender = tot_purchases_by_gender.rename(
    columns={"Price" : "Total Purchase Value"})

In [9]:
first_merge = pd.merge(tot_purchases_by_gender, avg_purchases_by_gender, on = "Gender", how="outer")
columns = ["Gender","Average Purchase Price", "Total Purchase Value"]
first_merge = first_merge[["Total Purchase Value", "Average Purchase Price"]]

In [10]:
second_merge = pd.merge(gender_purchases_df, first_merge, on="Gender", how="inner")

In [11]:
#Average Purchase Total per Person by Gender
avg_per_person = (second_merge[("Total Purchase Value")])/(gender_counts[("Total Count")])

In [12]:
second_merge["Avg Total Purchase per Person"] = avg_per_person.map("${0:,.2f}".format)
second_merge["Total Purchase Value"] = second_merge["Total Purchase Value"].map("${0:,.2f}".format)
second_merge["Average Purchase Price"] = second_merge["Average Purchase Price"].map("${0:,.2f}".format)

second_merge

,Gender,Purchase Count,Total Purchase Value,Average Purchase Price,Avg Total Purchase per Person
0,Female,113,$361.94,$3.20,$4.47
1,Male,652,"$1,967.64",$3.02,$4.07
2,Other / Non-Disclosed,15,$50.19,$3.35,$4.56


In [13]:
### Age Demographics

#The below each broken into bins of 4 years (i.e. &lt;10, 10-14, 15-19, etc.)
bins = [0, 9, 14, 19, 24, 29, 34, 39, 100]
age_group = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
 
age_info = purchase_data.loc[:, ("SN", "Age")]    
    
purchase_data["Age Group"] = pd.cut(age_info["Age"], bins, labels=age_group)
    
 # Purchase Count
purchases_by_age = purchase_data.groupby(["Age Group"]).size().reset_index(name="Purchase Count")
purchases_by_age

,Age Group,Purchase Count
0,<10,23
1,10-14,28
2,15-19,136
3,20-24,365
4,25-29,101
5,30-34,73
6,35-39,41
7,40+,13


In [14]:
# Average Purchase Price
avg_purchases_by_age = purchase_data.groupby(["Age Group"]).mean()
avg_purchases_by_age = avg_purchases_by_age.rename(
    columns={"Price" : "Average Purchase Price"})

In [15]:
# Total Purchase Value
tot_purchases_by_age = purchase_data.groupby(["Age Group"]).sum()
tot_purchases_by_age = tot_purchases_by_age.rename(
    columns={"Price" : "Total Purchase Value"})

In [16]:
age_count = age_info.drop_duplicates(subset = ["SN"], keep = "last", inplace = True)

purchase_data["Age Group"] = pd.cut(age_info["Age"], bins, labels=age_group)

purchases_by_age_group = purchase_data.groupby(["Age Group"]).size().reset_index(name="Total Count")
age_percent = (purchases_by_age_group["Total Count"]/players_count)*100
purchases_by_age_group["Percentage of Players"] = age_percent.map("{0:,.2f}%".format)
purchases_by_age_group

,Age Group,Total Count,Percentage of Players
0,<10,17,2.95%
1,10-14,22,3.82%
2,15-19,107,18.58%
3,20-24,258,44.79%
4,25-29,77,13.37%
5,30-34,52,9.03%
6,35-39,31,5.38%
7,40+,12,2.08%


In [17]:
age_merge = pd.merge(purchases_by_age, avg_purchases_by_age, on = "Age Group", how="outer")
age_merge_2 = pd.merge(age_merge, tot_purchases_by_age, on="Age Group", how="inner")
age_merge_2 = age_merge_2[["Age Group", "Total Purchase Value", "Average Purchase Price"]]

In [18]:
 # Average Purchase Total per Person by Age Group
avg_per_person_by_age = (age_merge_2[("Total Purchase Value")])/(purchases_by_age_group[("Total Count")])

In [19]:
age_merge_2["Avg Total Purchase per Person"] = avg_per_person_by_age.map("${0:,.2f}".format)
age_merge_2["Total Purchase Value"] = age_merge_2["Total Purchase Value"].map("${0:,.2f}".format)
age_merge_2["Average Purchase Price"] = age_merge_2["Average Purchase Price"].map("${0:,.2f}".format)

age_merge_2

,Age Group,Total Purchase Value,Average Purchase Price,Avg Total Purchase per Person
0,<10,$77.13,$3.35,$4.54
1,10-14,$82.78,$2.96,$3.76
2,15-19,$412.89,$3.04,$3.86
3,20-24,"$1,114.06",$3.05,$4.32
4,25-29,$293.00,$2.90,$3.81
5,30-34,$214.00,$2.93,$4.12
6,35-39,$147.67,$3.60,$4.76
7,40+,$38.24,$2.94,$3.19


In [29]:
### Top Spenders

#Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
 # SN
 # Purchase Count
 # Average Purchase Price
 # Total Purchase Value

spenders_list = purchase_data[["SN", "Price"]]
top_spenders = spenders_list.groupby(["SN"])
top_spenders_count = top_spenders["SN"].value_counts()
top_spenders_value = top_spenders.sum().sort_values("Price", ascending=False)
#top_spenders_value.head()
top_spenders_count.head()

SN            SN          
Adairialis76  Adairialis76    1
Adastirin33   Adastirin33     1
Aeda94        Aeda94          1
Aela59        Aela59          1
Aelaria33     Aelaria33       1
Name: SN, dtype: int64